In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Create dataset and dataloader

- we use FashionMNIST (see https://en.wikipedia.org/wiki/MNIST_database)
- the dataset is downloaded (if not already present)
- first we compute mean and std of the data over train dataset
- then we create new instance where normalization is applied

In [ ]:
temp_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
temp_loader = DataLoader(dataset=temp_dataset, batch_size=len(temp_dataset), shuffle=False)

# Compute mean and std
data_iter = iter(temp_loader)
images, _ = next(iter(data_iter))  # Get all images

mean, std = images.mean().item(), images.std().item()
print(f'Mean {mean}')
print(f'Mean {std}')

In [ ]:
# FashionMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

# Train model

- define a model as a MLP class (see below), define your own architecture
- use regularization, optimizer, loss,... of your choice and define hyperparameters so that you obtain high accuracy

In [ ]:
# Hyperparameters (add more)
batch_size = ...
learning_rate = ...
epochs = ...
hidden_sizes = ...

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# MLP Model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        ...

    def forward(self, x):
        ...

model = MLP().to(device)

In [ ]:
# Loss and optimizer
criterion = ...
optimizer = ...

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")